In [43]:
import os
import time
import random
import pandas as pnd
import BggDbGetter as BDG
from tqdm.notebook import tqdm
from boardgamegeek import BoardGameGeek
from boardgamegeek.exceptions import BoardGameGeekAPIError
from boardgamegeek.exceptions import BoardGameGeekAPIRetryError


### How does it work?
This class scrapes the pages found in BGG_latest.csv via the official API implemented into boardgamegeek library. <br>
It then proceeds to save them into another csv file called by default BGG_sampled.csv. <br>
The name of the csv to be created, the year range and the sampling rate can be specified.


In [46]:
class BggDbScraper:
    def __init__(self):
        self._csv = "BGG_latest.csv"
        self._bgg = BoardGameGeek()
        self._errors = []
    def db_scrape(self, date="2020-05-23", timeframe_start=2000, timeframe_end=2020, to_be_sampled=False, sampling_rate=1, final_name="BGG_sampled.csv"):
        if BDG.BggDbGetter().db_write(date):
            #BGG_latest.csv Dataframe reading and eventual sampling
            df = pnd.read_csv("BGG_latest.csv")
            df = df.loc[df["Year"].isin(range(timeframe_start, timeframe_end))]
            if to_be_sampled:
                df = df.sample(frac=sampling_rate).reset_index(drop=True)
            g = self._bgg.game(name=None, game_id=df.iloc[0]["ID"])
            
            cols = list(g.__dict__["_data"])
            new_df = pnd.DataFrame(columns = cols)
            new_df = new_df.drop(["thumbnail", "image", "expansion", "implementations", "expansions", "expands",\
                                  "alternative_names", "trading", "wanting", "wishing", "numcomments"], axis=1)            
            for row in tqdm(list(df.iterrows())):
                row_id = row[1]["ID"]
                new_row = {}
                try:
                    g = self._bgg.game(name=None, game_id=row_id)
                    for k, v in g.__dict__["_data"].items():
                        if k in new_df.columns:
                            new_row[k] = v                            
                    new_df = new_df.append(new_row, ignore_index=True)
                except (BoardGameGeekAPIError, BoardGameGeekAPIRetryError) as e:
                    print("Error, sleeping!")
                    time.sleep(0.5)
                    print("Error on ID: {0}".format(row_id))
                    self._errors.append(row_id)
                    while len(self._errors) > 0:
                        print("I am in while!")
                        err = self._errors.pop(0)
                        print("Err ID: ", err)
                        try:
                            print("I am in try n°2")
                            g = self._bgg.game(name=None, game_id=err)
                            for k, v in g.__dict__["_data"].items():
                                if k in new_df.columns:
                                    new_row[k] = v
                                new_df = new_df.append(new_row, ignore_index=True)
                        except (BoardGameGeekAPIError, BoardGameGeekAPIRetryError) as e2:
                            self._errors.append(err)
                            pass
                    pass
                #time.sleep(0.5)
            #new_df.drop_duplicates(subset="id", keep=False, inplace=True)
            new_df.to_csv(final_name)
        else:
            raise Exception("Error in BGG DB retrieval. Maybe you passed a non-existent date?")

In [47]:
#(list(df_sample.iterrows())[0][1]["ID"])
prova = BggDbScraper()
prova.db_scrape()


KeyboardInterrupt: 

### Reading of latest dataset

df = pnd.read_csv("BGG_latest.csv")
df.shape

### Selecting timeframe of 20 years to be analyzed

df = df.loc[df["Year"].isin(range(2000, 2020))]
for col in df.columns:
    print(col)
print("\nShape: ", df.shape)
df.loc[df["ID"] == 34084]
#df_sample = df.sample(frac=0.6).reset_index(drop=True)
#df_sample = df

### Construction of new dataframe with notable columns
<ul>
	<li>Eventual implementation of "Error_row", a list to correct the errors in retrieval phase</li>
</ul>

bgg = BoardGameGeek()
g = bgg.game(name=None, game_id=34084)

cols = list(g.__dict__["_data"])
new_df = pnd.DataFrame(columns = cols)
new_df = new_df.drop(["thumbnail", "image", "expansion", "implementations", "expansions", "expands", 
             "alternative_names", "trading", "wanting", "wishing", "numcomments"], axis=1)
error_row = []
#new_df

#### Retrieval of row informations via API request
<ul>
    <li>Some errors in retrieval phase pop up. Have to fix them, may do it in finale phase.</li>
    <li>Could clean it, may do it later.</li>
</ul>

for index, row in tqdm(list(df_sample.iterrows())):
    try:
        g = bgg.game(name=None, game_id=row["ID"])
        new_row = {}
        for k, v in g.__dict__["_data"].items():
            if k in new_df.columns:
                #print("{0}: {1}".format(k, v))
                new_row[k] = v
        new_df = new_df.append(new_row, ignore_index=True)
    except:
        print("Error on iteration row {0}".format(index))
        #error_row.append(df_sample.iloc[index])
        g = bgg.game(name=None, game_id=row["ID"])
        new_row = {}
        for k, v in g.__dict__["_data"].items():
            if k in new_df.columns:
                #print("{0}: {1}".format(k, v))
                new_row[k] = v
        new_df = new_df.append(new_row, ignore_index=True)
        pass
    time.sleep(0.5)

##### Correct retriever, modified in upper cell in order to try something more complex eventually. Make it "code" to try it

for index, row in tqdm(list(df_sample.iterrows())):
    try:
        g = bgg.game(name=None, game_id=row["ID"])
        new_row = {}
        for k, v in g.__dict__["_data"].items():
            if k in new_df.columns:
                #print("{0}: {1}".format(k, v))
                new_row[k] = v
        new_df = new_df.append(new_row, ignore_index=True)
    except:
        print("Error on iteration row {0}".format(index))
        error_row.append(df_sample.iloc[index])
        pass
    time.sleep(0.5)

##### Stub error solver, make it "code" to try it
while len(error_row) > 0:
    try:
        g = bgg.game(name=None, game_id=error_row[i])
        new_row = {}
        for k, v in g.__dict__["_data"].items():
            if k in new_df.columns:
                new_row[k] = v
        new_df = new_df.append(new_row, ignore_index=True)
        error_row.pop(0)
    except:
        #print("Error on iteration row {0}".format(index))
        error_row.append(df_sample.iloc[index])
        pass

##### Dataframe cleaner from duplicates and saving to csv

new_df.drop_duplicates(subset="id", keep=False, inplace=True)
new_df.to_csv("./BGG_sampled.csv")

## TODO: Make a python class in order to handle all this operation cleanly